In [ ]:
import random
import tensorflow as tf
import numpy as np
import similarity

%run utils.py

datadir = pathlib.Path("/home/david/pro/scc/data")

In [ ]:
relevant = tuple(year for year in (1845, 1850, 1860, 1880, 1885))

In [ ]:
with (datadir / ".." / "links" / "linkede_personer.csv").open("r", encoding="latin1") as fd:
    print(next(fd))
    links = {year: {} for year in relevant} # year -> (kip,løb) -> linkID
    for line in fd:
        split = [part.lower() for part in line.strip().split("|")]
        for year in relevant:
            if str(year) in str(split[1]):
                kipløb = tuple(split[2:4])
                links[year][kipløb] = split[0]
                break

In [ ]:
for year, matched in links.items():
    print(year, len(matched))

TODO: vælg bedre subset til at være ikke-matches

In [ ]:
linked_data = {} # linkID -> [(year, row)]
unmatches = []
for fn in sorted(datadir.glob("lc_*.csv")):
    year = re.search(r"\d{4}", fn.name).group(0)
    year_num = int(year)
    print(year_num)
    current_links = links[year_num]
    
    with fn.open("r", encoding="UTF-8") as fd:
        print(next(fd))
        
        for line in fd:
            entry = parseEntry(line)
            if random.random() < 0.01:
                unmatches.append(entry)
            løb = entry.løbenr.split(",")[0]
            pair = (entry.kipnr, løb)
            if pair in current_links:
                linkID = current_links[pair]
                linked_data.setdefault(linkID, []).append((year_num, entry))

In [ ]:
useless = [key for key,val in linked_data.items() if len(val) == 1]
for key in useless:
    del(linked_data[key])

In [ ]:
def get_score(a, b):
    name_scores = (similarity.string_linear(getattr(a, prop), getattr(b, prop))
                   for prop in ("fornavn", "mellemnavn", "efternavn", "initialer"))
    name_scores_alt = (similarity.string_inverse(getattr(a, prop), getattr(b, prop))
                   for prop in ("fornavn", "mellemnavn", "efternavn", "initialer"))
    geo_scores = (similarity.string_linear(a.fødested, b.fødested),
                   similarity.geo(a.fødested, b.fødested))
    job_scores = (similarity.string_linear(getattr(a, prop), getattr(b, prop))
                   for prop in ("civilstand", "position"))
    state_score = similarity.status(a.civilstand, b.civilstand)
    age_score = similarity.birthyear(a.fødeår, b.fødeår)
    return np.array((*name_scores, *name_scores_alt, *geo_scores, *job_scores, state_score, age_score))

In [ ]:
input_size = 14
output_size = 2

In [ ]:
# input
x = tf.placeholder(tf.float32, [None, input_size], name="x-in-data")

W = tf.Variable(tf.zeros([input_size, output_size]))
b = tf.Variable(tf.zeros([output_size]))

# output layer
y = tf.nn.softmax(tf.matmul(x, W) + b)

# now declare the output data placeholder
y_ = tf.placeholder(tf.float32, [None, output_size], name="y-out-data")

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
s = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
for i in range(100):
    data = []
    labels = []

    for linkID in random.sample(list(linked_data), 100):
        rows = linked_data[linkID]
        for (a_year, a), (b_year, b) in zip(rows[:-1], rows[1:]):
            data.append(get_score(a, b))
            labels.append((0, 1))

    for i in range(100):
        a, b = random.sample(unmatches, 2)
        data.append(get_score(a, b))
        labels.append((1, 0))

    x_train = np.vstack(data)
    y_train = np.vstack(labels)
    s.run(train_step, feed_dict={x: x_train, y_: y_train})

In [ ]:
s.run(accuracy, feed_dict={x: x_train, y_: y_train})

In [ ]:
s.run(W)

De skulle være matches; forvent [1, 1, 1, ...]

In [ ]:
np.argmax(s.run(y, feed_dict={x: data[:10]}), axis=1)

In [ ]:
s.run(y, feed_dict={x: data[:10]})

De skulle ikke; forvent [0, 0, 0, ...]

In [ ]:
np.argmax(s.run(y, feed_dict={x: data[-10:]}), axis=1)

In [ ]:
y_train

In [ ]:
np.argmax(out, axis=1)

# TEST

In [ ]:
subset = {}
for fn in sorted(datadir.glob("lc_*.csv")):
    year = re.search(r"\d{4}", fn.name).group(0)
    year_num = int(year)
    print(year_num)
    current_set = subset[year_num] = []
    
    with fn.open("r", encoding="UTF-8") as fd:
        print(next(fd))
        
        for line in fd:
            entry = parseEntry(line)
            if entry.initialer == "dh":
                current_set.append(entry)

In [ ]:
for year, entries in subset.items():
    print(year, len(entries))

In [ ]:
find_me = random.choice(subset[1845])

In [ ]:
find_me

In [ ]:
import operator

In [ ]:
results = []
for entry in subset[1850]:
    score = get_score(find_me, entry)
    value = s.run(y, feed_dict={x: np.vstack([score])})
    if value[0][1] > 0.8:
        results.append((value[0][1], entry))
results.sort(key=operator.itemgetter(0), reverse=True)

In [ ]:
results